In [2]:
import pandas as pd
import os
import geopandas as gpd
import geoplot as gplt
import scipy as sp
from scipy.spatial import Voronoi, voronoi_plot_2d
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon
import sys

In [ ]:
wd = '/home/beeb/Insync/sei2112@columbia.edu/Google Drive/Columbia SusDev PhD/Research/My work/predictArsenic/'
placesPath = 'data/raw/googlePlaces/results.txt'
shapePath = 'data/raw/shapefiles/bgd_admbnda_adm{}_bbs_20180410/'
outPath = 'data/intermediate/placesToShapes/results.shp'
figPath = 'output/intermediate/placesToShapes/plot.png'
os.chdir(wd)

In [ ]:
# read in data
dat = pd.read_json(placesPath)

In [ ]:
# extract latitudes, longitudes and names
lats = [result[0]['geometry']['location']['lat'] for result in dat['results']]
lons = [result[0]['geometry']['location']['lng'] for result in dat['results']]
names = [result[0]['name'] for result in dat['results']]
formatted_address = [result[0]['formatted_address'] for result in dat['results']]

In [ ]:
# put these into a geodataframe
df = pd.DataFrame(data = {'longitude': lons, 'latitude': lats},#, 'formatted_address': formatted_address}, 
                  index = names).drop_duplicates()
#gdf = gpd.GeoDataFrame(df, geometry = gpd.points_from_xy(df.longitude, df.latitude))
#gdf.head()

In [ ]:
# read in the Bangladesh shapefile

In [ ]:
shp = gpd.read_file(shapePath.format(0))

In [ ]:
shp.plot()

In [ ]:
# filter out any entries in our gdf which are outside Bangladesh, check 
# this doesn't involve dropping > 10% of obs
beginPointsN = df.shape[0]
df = df[df.latitude <= shp.bounds.loc[0, 'maxy']]
df = df[df.latitude >= shp.bounds.loc[0, 'miny']]
df = df[df.longitude < shp.bounds.loc[0, 'maxx']]
df = df[df.longitude >= shp.bounds.loc[0, 'minx']]
endPointsN = df.shape[0]
assert endPointsN > 0.9 * beginPointsN

In [ ]:
df.head()

In [ ]:
#CODE FROM THE INTERWEBS 
# this code creates a voronoi diagram within a bounding box
#https://stackoverflow.com/questions/28665491/getting-a-bounded-polygon-coordinates-from-voronoi-cells

eps = sys.float_info.epsilon

bounding_box = shp.bounds.loc[0, ['minx', 'maxx', 'miny', 'maxy']] # [x_min, x_max, y_min, y_max]

def in_box(towers, bounding_box):
    return np.logical_and(np.logical_and(bounding_box[0] <= towers[:, 0],
                                         towers[:, 0] <= bounding_box[1]),
                          np.logical_and(bounding_box[2] <= towers[:, 1],
                                         towers[:, 1] <= bounding_box[3]))


def voronoi(towers, bounding_box):
    # Select towers inside the bounding box
    i = in_box(towers, bounding_box)
    # Mirror points
    points_center = towers[i, :]
    points_left = np.copy(points_center)
    points_left[:, 0] = bounding_box[0] - (points_left[:, 0] - bounding_box[0])
    points_right = np.copy(points_center)
    points_right[:, 0] = bounding_box[1] + (bounding_box[1] - points_right[:, 0])
    points_down = np.copy(points_center)
    points_down[:, 1] = bounding_box[2] - (points_down[:, 1] - bounding_box[2])
    points_up = np.copy(points_center)
    points_up[:, 1] = bounding_box[3] + (bounding_box[3] - points_up[:, 1])
    points = np.append(points_center,
                       np.append(np.append(points_left,
                                           points_right,
                                           axis=0),
                                 np.append(points_down,
                                           points_up,
                                           axis=0),
                                 axis=0),
                       axis=0)
    # Compute Voronoi
    vor = sp.spatial.Voronoi(points)
    # Filter regions
    regions = []
    for region in vor.regions:
        flag = True
        for index in region:
            if index == -1:
                flag = False
                break
            else:
                x = vor.vertices[index, 0]
                y = vor.vertices[index, 1]
                if not(bounding_box[0] - eps <= x and x <= bounding_box[1] + eps and
                       bounding_box[2] - eps <= y and y <= bounding_box[3] + eps):
                    flag = False
                    break
        if region != [] and flag:
            regions.append(region)
    vor.filtered_points = points_center
    vor.filtered_regions = regions
    return vor


In [ ]:
# make a voronoi diagram and plot it
points = np.array([[df.longitude.iloc[i], df.latitude.iloc[i]] for i in range(df.shape[0])])
vor = voronoi(points, bounding_box)
fig = voronoi_plot_2d(vor)
plt.show()

In [ ]:
# wild guess at how to do this
regions = [region for region in vor.filtered_regions if len(region) > 0]

# function for turning the voronoi objects into polygons
def getPoly(pointIndices):
    
    # get the vertices for this polygon
    poly = vor.vertices[pointIndices,:]
    
    # turn this into a polygon object
    poly = Polygon(poly)
    
    # check the polygon is valid (i.e. it does not cross itself)
    if not poly.is_valid:
        print('Invalid')
        
    # give it the CRS that Google uses
    poly.crs = 'EPSG:4326'
    return(poly)

polygons = [getPoly(pointIndex) for pointIndex in regions]# if len(pointIndex) > 0]
#df['vor'] = vor.point_region

In [ ]:
# check that all resulting polygons are valid
assert sum([not x.is_valid for x in polygons]) == 0

# turn the polygons into a geoseries and set the CRS
polydf = gpd.GeoSeries(polygons)
polydf.crs = 'EPSG:4326'

# clip to Bangladesh shape
polydf = gpd.clip(polydf, shp)

In [ ]:
# add in the area name to each of the polygons
# let's hope that Python doesn't mix all these polygons around so they're all in the same order???
polydf.index = df.index

In [ ]:
polydf.to_file(outPath)

In [ ]:
polydf.head()

In [ ]:
polydf.plot()
plt.savefig(figPath)